# Influential data identification - Llama2 - Math - Reason

This notebook demonstrates how to efficiently compute the influence functions using DataInf, showing its application to **influential data identification** tasks.

- Model: [llama-2-13b-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) trained on a mix of publicly available online datasets.
- Fine-tuning dataset: Synthetic Math Problem (with reasoning) dataset

References
- `trl` HuggingFace library [[Link]](https://github.com/huggingface/trl).
- DataInf is available at this [ArXiv link](https://arxiv.org/abs/2310.00902).

In [1]:
import sys
sys.path.append('../src')
from lora_model import LORAEngineGeneration
from influence import IFEngineGeneration

import warnings
warnings.filterwarnings("ignore")

/kuacc/users/oince22/.conda/envs/datainf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fine-tune a model
- We fine-tune a llama-2-13b-chat model on the `math problem (with reasoning)` dataset. We use `src/sft_trainer.py`, which is built on HuggingFace's [SFTTrainer](https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py). It will take around 30 minutes.
- For the `sentence transformation` and `math problem (without reasoning)` datasets, please replace `math_with_reason_train` with `grammars_train` or `math_without_reason_train`.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [3]:
!python /kuacc/users/oince22/hpc_run/DataInf/src/sft_trainer.py \
    --model_name /kuacc/users/oince22/hpc_run/OPT/llama-2-7b-chat \
    --dataset_name /kuacc/users/oince22/hpc_run/DataInf/datasets/math_with_reason_train.hf \
    --output_dir /kuacc/users/oince22/hpc_run/DataInf/models/math_with_reason_7bf \
    --dataset_text_field text \
    --load_in_8bit \
    --use_peft

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/kuacc/users/oince22/.conda/envs/datainf/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Traceback (most recent call last):
  File "/kuacc/users/oince22/hpc_run/DataInf/src/sft_trainer.py", line 88, in <module>
    model = AutoModelForCausalLM.from_pretrained(
  File "/kuacc/users/oince22/.conda/envs/datainf/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 487, in from_pretrained
    resolved_config_file = cached_file(
  File "/kuacc/users/oince22/.conda/envs/datainf/lib/python3.10/site-packages/transformers/utils/hub.py", line 429, in cached_file
    resolved_file = hf_hub_download(
  File "/kuacc/users/oince22/.conda/envs/dat

## Load a fine-tuned model

In [3]:
# Please change the following objects to  "YOUR-LLAMA-PATH" and "YOUR-DATAINF-PATH"
base_path = "/kuacc/users/oince22/hpc_run/OPT/llama-2-7b-chat" 
project_path ="/kuacc/users/oince22/hpc_run/DataInf" 
lora_engine = LORAEngineGeneration(base_path=base_path, 
                                   project_path=project_path,
                                   dataset_name='math_with_reason')

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Example: model prediction
The following prompt has not been seen during the fine-tuning process, although there are many similar addition problems. 

In [4]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
prompt = """
Emily scored 10 points in the first game, 30 points in the second, 100 in the third, and 20 in the fourth game. What is her total points? Output only the answer.
"""
inputs = lora_engine.tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate
generate_ids = lora_engine.model.generate(input_ids=inputs.input_ids, 
                                          max_length=128,
                                          pad_token_id=lora_engine.tokenizer.eos_token_id)
output = lora_engine.tokenizer.batch_decode(
    generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

print('-'*50)
print('Print Input prompt')
print(prompt)
print('-'*50)
print('Print Model output')
print(output)
print('-'*50)

--------------------------------------------------
Print Input prompt

Emily scored 10 points in the first game, 30 points in the second, 100 in the third, and 20 in the fourth game. What is her total points? Output only the answer.

--------------------------------------------------
Print Model output

Emily scored 10 points in the first game, 30 points in the second, 100 in the third, and 20 in the fourth game. What is her total points? Output only the answer.

Answer: 160
--------------------------------------------------


## Compute the gradient
 - Influence function uses the first-order gradient of a loss function. Here we compute gradients using `compute_gradient`
 - `tr_grad_dict` has a nested structure of two Python dictionaries. The outer dictionary has `{an index of the training data: a dictionary of gradients}` and the inner dictionary has `{layer name: gradients}`. The `val_grad_dict` has the same structure but for the validationd data points. 

In [6]:
tokenized_datasets, collate_fn = lora_engine.create_tokenized_datasets()
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn)

Parameter 'function'=<function LORAEngineGeneration.create_tokenized_datasets.<locals>.<lambda> at 0x15543fe432e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:04<00:00,  1.54it/s]


## Compute the influence function
 - We compute the inverse Hessian vector product first using `compute_hvps()`. With the argument `compute_accurate=True`, the exact influence function value will be computed. (it may take an hour to compute).
<!--  - Here, we take a look at the first five validation data points. -->

In [7]:
influence_engine = IFEngineGeneration()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict)
influence_engine.compute_hvps()
influence_engine.compute_IF()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [36:18<00:00, 21.79s/it]


Computing IF for method:  identity
Computing IF for method:  proposed


## Attributes of influence_engine
There are a couple of useful attributes in `influence_engine`. For intance, to compare the runtime, one can use `time_dict`.

In [8]:
influence_engine.time_dict

defaultdict(list,
            {'identity': 3.814697265625e-06, 'proposed': 2178.7828104496})

In [9]:
influence_engine.IF_dict.keys()

dict_keys(['identity', 'proposed'])

## Application to influential data detection task
- We inspect the most and the least influential data points for validation data loss. Here, the most (and the least) influential data points are determined by the absolute value of influence function values.
- Why? the least influential data points will have near zero values, which means the training data point does not affect the validation loss. 

In [10]:
most_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmax(), axis=1)
least_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmin(), axis=1)

In [11]:
val_id=21
print(f'Validation Sample ID: {val_id}\n', 
      lora_engine.validation_dataset[val_id]['text'], '\n')
print('The most influential training sample: \n', 
      lora_engine.train_dataset[int(most_influential_data_point_proposed.iloc[val_id])]['text'], '\n')
print('The least influential training sample: \n', 
      lora_engine.train_dataset[int(least_influential_data_point_proposed.iloc[val_id])]['text'])

Validation Sample ID: 21
 Solve the following math problem. In an aquarium, there are 67 sharks and 11 dolphins. If they bought 2 more sharks, how many sharks would be there in total? -> Reason: Total sharks = 67 + 2. Answer: 69</s> 

The most influential training sample: 
 Solve the following math problem. In an aquarium, there are 79 sharks and 56 dolphins. If they bought 2 more sharks, how many sharks would be there in total? -> Reason: Total sharks = 79 + 2. Answer: 81</s> 

The least influential training sample: 
 Solve the following math problem. Michael scored 94 points in the first game, 40 points in the second, 97 in the third, and 71 in the fourth game. What is his total points? -> Reason: Total points = 94 + 40 + 97 + 71. Answer: 302</s>


# AUC and Recall 

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

identity_df=influence_engine.IF_dict['identity']
proposed_df=influence_engine.IF_dict['proposed']

n_train, n_val = 900, 100
n_sample_per_class = 90 
n_class = 10

identity_auc_list, proposed_auc_list=[], []
for i in range(n_val):
    gt_array=np.zeros(n_train)
    gt_array[(i//n_class)*n_sample_per_class:((i//n_class)+1)*n_sample_per_class]=1
    
    # The influence function is anticipated to have a big negative value when its class equals to a validation data point. 
    # This is because a data point with the same class is likely to be more helpful in minimizing the validation loss.
    # Thus, we multiply the influence function value by -1 to account for alignment with the gt_array. 
    identity_auc_list.append(roc_auc_score(gt_array, -(identity_df.iloc[i,:].to_numpy())))
    proposed_auc_list.append(roc_auc_score(gt_array, -(proposed_df.iloc[i,:].to_numpy())))
    
print(f'identity AUC: {np.mean(identity_auc_list):.3f}/{np.std(identity_auc_list):.3f}')
print(f'proposed AUC: {np.mean(proposed_auc_list):.3f}/{np.std(proposed_auc_list):.3f}')

identity AUC: 0.772/0.173
proposed AUC: 1.000/0.001


In [13]:
# Recall calculations
identity_recall_list, proposed_recall_list=[], []
for i in range(n_val):
    correct_label = i // 10

    # Similar to AUC computation, we consider the first 90 data points with the smallest influence function values 
    # These data points with the smallest influence function values likely have the same class with the validation data point.
    sorted_labels = np.argsort(identity_df.iloc[i].values)// 90 
    recall_identity = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    identity_recall_list.append(recall_identity)
    
    sorted_labels = np.argsort(proposed_df.iloc[i].values)// 90 
    recall_proposed = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    proposed_recall_list.append(recall_proposed)
    
print(f'identity Recall: {np.mean(identity_recall_list):.3f}/{np.std(identity_recall_list):.3f}')
print(f'proposed Recall: {np.mean(proposed_recall_list):.3f}/{np.std(proposed_recall_list):.3f}')

identity Recall: 0.258/0.388
proposed Recall: 0.996/0.025
